# ABS Detailed Labour Force 6291

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from plotting import (
    finalise_plot,
    set_chart_dir,
    recalibrate_series,
    clear_chart_dir,
    line_plot,
)
from abs_data_capture import (
    get_fs_constants,
    get_ABS_meta_and_data,
    get_plot_constants,
    get_identifier,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999

# show charts within this notebook
SHOW = False

In [2]:
## Get data from ABS

In [3]:
catalogue_id = "6291"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

# --- monthly
abs_m = get_ABS_meta_and_data(catalogue_id)
if abs_m is None:
    sys.exit(-1)
meta_m = abs_m[META_DATA]
RECENT_M, plot_times, plot_tags = get_plot_constants(meta_m)
print(f"Monthly tables: {meta_m['Table'].unique()}")

# --- quarterly
abs_q = get_ABS_meta_and_data(catalogue_id, 2)
if abs_q is None:
    sys.exit(-1)
meta_q = abs_q[META_DATA]
RECENT_Q, plot_times_q, plot_tags_q = get_plot_constants(meta_q)
print(f"Quarterly tables: {meta_q['Table'].unique()}")


Found URL for a ZIP file on ABS web page
We need to download this file from the ABS ...
Saving ABS download to cache.
Extracting DataFrames from the zip-file ...
Monthly tables: ['01' '02' '03' '08' '09' '10' '14a' '14b' '14c' '14d' '14e' '16' '16b'
 '16c' '20a' '20b' '20c']
Found URL for a ZIP file on ABS web page
We need to download this file from the ABS ...
Saving ABS download to cache.
Extracting DataFrames from the zip-file ...
Quarterly tables: ['04' '05' '06' '07' '11' '12' '13' '17' '19' '23a' '23b' '24a' '24b'
 '25a' '25b' '26a' '26b' '27' '28' '29a' '29b']


## Plot

In [4]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Monthly by Age

In [5]:
table = "01"
data = abs_m[table]
series_type = "Original"

ages = [
    "15-24 years",
    "25-34 years",
    "35-44 years",
    "45-54 years",
    "55-64 years",
]

TOKEN = "By Age"
plots = [
    f"> {TOKEN} ;  Participation rate ;  Persons ;",
    f"> {TOKEN} ;  Unemployment rate ;  Persons ;",
]

for plot in plots:
    combined = {}
    for age in ages:
        desc = plot.replace(TOKEN, age)
        ident, units = get_identifier(meta_m, desc, series_type, table)
        combined[age] = data[ident]

    title = f"Australia: {plot}"
    line_plot(
        pd.DataFrame(combined),
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter=f"{series_type.capitalize()} series",
        show=SHOW,
    )

### Quarterly by sector

In [6]:
table = "04"
data_table = abs_q[table]
series_type = "Seasonally Adjusted"
selection = meta_q[(meta_q["Series Type"] == series_type) & (meta_q["Table"] == table)]

for index, series in selection.iterrows():
    title = series["Data Item Description"].replace(" ;", "").replace("  ", " ")
    id, units = series["Series ID"], series["Unit"]
    series, units = recalibrate_series(data_table[id], units)
    line_plot(
        series,
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter=f"{series_type.capitalize()} series",
        show=SHOW,
    )

### Monthly by duration of unemployment

In [7]:
table = "14a"
df = abs_m[table]
series_type = "Original"

cohorts = [
    "2 years and over",
    "1-2 years",
    "6-12 months",
    "3-6 months",
    "1-3 months",
    "under 1 month",
]

categories = ["Unemployed total ;  Persons ;"]

did = "Data Item Description"
period = 12
for category in categories:
    ax = None
    raw = pd.DataFrame()
    trend = pd.DataFrame()
    for cohort in cohorts:
        rows = meta_m[
            meta_m[did].str.contains(category) & meta_m[did].str.contains(cohort)
        ]
        assert len(rows) == 1
        select = rows["Series ID"].values[0]
        units = rows["Unit"].values[0]
        raw[cohort] = df[select]
        trend[cohort] = df[select].rolling(period).mean()

    title = category.replace(" ; ", " ").replace(" ;", " ").replace("  ", " ").title()
    lfooter = f"Australia. {series_type.capitalize()} series. "

    for data, t in zip([raw, trend], [title, f"{title}, {period} month ave."]):
        line_plot(
            data,
            title=t,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            show=SHOW,
        )

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Sep 27 2023 09:03:33

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

matplotlib: 3.8.0
sys       : 3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:35:23) [Clang 15.0.7 ]
pandas    : 2.1.1

Watermark: 2.4.3



In [9]:
print("Finished")

Finished
